# Fama-French Data

In [1]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np

start_date = "1960-01-01"
end_date = "2022-12-31"

factors_ff3_monthly_raw = pdr.DataReader(
  name="F-F_Research_Data_Factors",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

factors_ff3_monthly = (factors_ff3_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

In [ ]:
factors_ff5_monthly_raw = pdr.DataReader(
  name="F-F_Research_Data_5_Factors_2x3",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

factors_ff5_monthly = (factors_ff5_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_9840/2113571173.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff5_monthly_raw = pdr.DataReader(
/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_9840/2113571173.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff5_monthly_raw = pdr.DataReader(


In [ ]:
factors_ff3_daily_raw = pdr.DataReader(
  name="F-F_Research_Data_Factors_daily",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

factors_ff3_daily = (factors_ff3_daily_raw
  .divide(100)
  .reset_index(names="date")
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_9840/4051783287.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factors_ff3_daily_raw = pdr.DataReader(


In [ ]:
industries_ff_monthly_raw = pdr.DataReader(
  name="10_Industry_Portfolios",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

industries_ff_monthly = (industries_ff_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
)

/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_9840/192540579.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_monthly_raw = pdr.DataReader(
/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_9840/192540579.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_monthly_raw = pdr.DataReader(
/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_9840/192540579.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_monthly_raw = pdr.DataReader(
/var/folders/_y/0xw7_0ss0

# q-Factors

In [ ]:
factors_q_monthly_link = (
  "https://global-q.org/uploads/1/2/2/6/122679606/"
  "q5_factors_monthly_2022.csv"
)

factors_q_monthly = (pd.read_csv(factors_q_monthly_link)
  .assign(
    month=lambda x: (
      pd.to_datetime(x["year"].astype(str) + "-" +
        x["month"].astype(str) + "-01"))
  )
  .drop(columns=["R_F", "R_MKT", "year"])
  .rename(columns=lambda x: x.replace("R_", "").lower())
  .query(f"month >= '{start_date}' and month <= '{end_date}'")
  .assign(
    **{col: lambda x: x[col]/100 for col in ["me", "ia", "roe", "eg"]}
  )
)

# Macroeconomic Predictors

In [ ]:
sheet_id = "1g4LOaRj4TvwJr9RIaA_nwrXXWTOy46bP"
sheet_name = "macro_predictors.xlsx"
macro_predictors_link = (
  f"https://docs.google.com/spreadsheets/d/{sheet_id}" 
  f"/gviz/tq?tqx=out:csv&sheet={sheet_name}"
)

macro_predictors = (
  pd.read_csv(macro_predictors_link, thousands=",")
  .assign(
    month=lambda x: pd.to_datetime(x["yyyymm"], format="%Y%m"),
    dp=lambda x: np.log(x["D12"])-np.log(x["Index"]),
    dy=lambda x: np.log(x["D12"])-np.log(x["D12"].shift(1)),
    ep=lambda x: np.log(x["E12"])-np.log(x["Index"]),
    de=lambda x: np.log(x["D12"])-np.log(x["E12"]),
    tms=lambda x: x["lty"]-x["tbl"],
    dfy=lambda x: x["BAA"]-x["AAA"]
  )
  .rename(columns={"b/m": "bm"})
  .get(["month", "dp", "dy", "ep", "de", "svar", "bm", 
        "ntis", "tbl", "lty", "ltr", "tms", "dfy", "infl"])
  .query("month >= @start_date and month <= @end_date")
  .dropna()
)

In [ ]:
cpi_monthly = (pdr.DataReader(
    name="CPIAUCNS", 
    data_source="fred", 
    start=start_date, 
    end=end_date
  )
  .reset_index(names="month")
  .rename(columns={"CPIAUCNS": "cpi"})
  .assign(cpi=lambda x: x["cpi"]/x["cpi"].iloc[-1])
)

# Set Database

In [ ]:
import os
import sqlite3

# Create the data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Provide the full path to the SQLite database file
db_file = os.path.join(data_dir, "tidy_finance_python.sqlite")

# Connect to the SQLite database (this will create the database file if it doesn't exist)
tidy_finance = sqlite3.connect(db_file)


In [ ]:
import sqlite3
tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")

(factors_ff3_monthly
  .to_sql(name="factors_ff3_monthly", 
          con=tidy_finance, 
          if_exists="replace",
          index=False)
)

pd.read_sql_query(
  sql="SELECT month, rf FROM factors_ff3_monthly",
  con=tidy_finance,
  parse_dates={"month"}
)

,month,rf
0,1960-01-01,0.0033
1,1960-02-01,0.0029
2,1960-03-01,0.0035
3,1960-04-01,0.0019
4,1960-05-01,0.0027
...,...,...
751,2022-08-01,0.0019
752,2022-09-01,0.0019
753,2022-10-01,0.0023
754,2022-11-01,0.0029


In [ ]:
data_dict = {
  "factors_ff5_monthly": factors_ff5_monthly,
  "factors_ff3_daily": factors_ff3_daily,
  "industries_ff_monthly": industries_ff_monthly, 
  "factors_q_monthly": factors_q_monthly,
  "macro_predictors": macro_predictors,
  "cpi_monthly": cpi_monthly
}

for key, value in data_dict.items():
    value.to_sql(name=key,
                 con=tidy_finance, 
                 if_exists="replace",
                 index=False)
    
